In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import f1_score
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
import seaborn as sns
#from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU, BatchNormalization
#from tensorflow.keras import layers, initializers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras import layers, models

In [2]:
model_path = "06d01m2024ylr4grayep50.h5" #model name of which model you want to load

In [3]:
from tensorflow.keras.models import load_model
model = load_model(os.path.join('models',model_path))

In [4]:
path  = datasetpath
def process_img(img, size = (224,224)):
    img = cv2.resize(img, size)
    img = img/255
    return img 

NameError: name 'datasetpath' is not defined

In [ ]:
def histogram_equalization(image, size = (224,224)):
    image = cv2.resize(image, size)
    equalized_image = cv2.equalizeHist(image)
    equalized_image = equalized_image/255
    return equalized_image

In [ ]:
def compute_morphological_gradient(image, kernel_size=3, size = (224,224)):
    image = cv2.resize(image, size)
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    dilation = cv2.dilate(image, kernel, iterations=5)
    erosion = cv2.erode(image, kernel, iterations=1)
    gradient_image = cv2.subtract(dilation, erosion)
    gradient_image = gradient_image/255
    return gradient_image

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def calculate_dtm(image, threshold=28, size = (224, 224)):
    # Görüntüyü yükle
    input_image = cv2.resize(image, size)

    # Sobel türevlerini hesapla
    sobelx = cv2.Sobel(input_image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(input_image, cv2.CV_64F, 0, 1, ksize=3)

    # Gradient magnitude hesapla
    dtm = np.sqrt(sobelx**2 + sobely**2)

    # Eşikleme uygula
    dtm_thresholded = np.where(dtm > threshold, 255, 0).astype(np.uint8)
    dtm_thresholded = dtm_thresholded/255
    return dtm_thresholded

In [ ]:
import cv2
import numpy as np
from skimage import exposure
from skimage import feature
import matplotlib.pyplot as plt

def compute_hog(image, pixels_per_cell=(16, 16), size = (224, 224)):
    image = cv2.resize(image, size)
    # Compute HOG features with higher resolution
    hog_features, hog_image = feature.hog(image, visualize=True, pixels_per_cell=pixels_per_cell)
    # Enhance the contrast of the HOG image
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 28))

    return hog_image_rescaled

In [ ]:
def choose(name, image):
    if name=="gray":
        return process_img(image)
    else if name=="histo":
        return histogram_equalization(image)
    else if name=="morpho":
        return compute_morphological_gradient(image)
    else if name=="dtm":
        return calculate_dtm(image)
    else if name=="hog":
        return compute_hog(image)

In [ ]:
choice = "gray" #please choose correct operation that model is trained for.
#gray/histo/morpho/dtm/hog

In [ ]:
path = datasetpath
images, values = [],[]
for name in os.listdir(path):
    for name2 in os.listdir(os.path.join(path, name)):
        if not name2.startswith('.'):
            img = cv2.imread(os.path.join(path, name, name2), cv2.IMREAD_GRAYSCALE)
            images.append(choose(choice,img))
            values.append(name.replace(', ', '_'))
            
more_images, more_values = images, values

from sklearn.preprocessing import LabelEncoder
# Sample data with string labels
orijinal_liste = more_values
labels = sorted(list(set(orijinal_liste)))
print(labels)
label_encoder = LabelEncoder()
numerical_labels = label_encoder.fit_transform(labels)
for label, numerical_label in zip(labels, numerical_labels):
    print(f'{label} -> {numerical_label}')
arr = []
for i in more_values:
    arr.append(labels.index(i))
more_values = arr

more_images = np.array(more_images)
more_values = np.array(more_values) 

more_images = np.repeat(more_images[..., np.newaxis], 3, -1)


x_train, x_test, y_train, y_test = train_test_split(more_images, arr, test_size=0.5, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.6, random_state=42)

y_val = np.array(y_val)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
model.evaluate(x_test,y_test)

from tensorflow.keras.utils import to_categorical
# One-hot encode y_test
y_test_hot = to_categorical(y_test, num_classes=len(labels))

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test_hot, axis=1)

f1 = f1_score(np.argmax(y_test_hot, axis=1), y_pred_classes, average='weighted')
print("F1-score: ", f1)

confusion_mtx = confusion_matrix(y_test_classes, y_pred_classes)
plt.figure(figsize=(12,10))
sns.heatmap(confusion_mtx, annot=True, xticklabels=labels, yticklabels=labels)
plt.title('Normalized Confusion Matrix')
plt.show()

fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

fig = plt.figure()
plt.plot(history.history['sparse_categorical_accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_sparse_categorical_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()